In [1]:
import os
os.getcwd()

'/Users/rutu/Desktop/INSTACART'

In [2]:
import pandas as pd

In [3]:
prior = pd.read_csv('order_products__prior.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [4]:
# orders in prior merged with product names
prior_products = pd.merge(prior, products, how='left', on='product_id')


In [5]:
# Prior orders with user_id, product_id, product_name
prior_products_order = pd.merge(prior_products, orders, how='left', on='order_id')

In [6]:
prior_products_order['product_name'][0]

'Organic Egg Whites'

In [6]:
#Sep = _ for all product name
products = prior_products_order['product_name']
product_no_space = []
for product in products:
    product = product.replace(" ", "_")
    product_no_space.append(product)


In [7]:
#Repalcing the oldcol with the new one(with _)
prior_products_order.drop(['product_name'], axis=1)
prior_products_order['product_name'] = product_no_space

In [9]:
print(prior_products_order)

          order_id  product_id  add_to_cart_order  reordered  \
0                2       33120                  1          1   
1                2       28985                  2          1   
2                2        9327                  3          0   
3                2       45918                  4          1   
4                2       30035                  5          0   
5                2       17794                  6          1   
6                2       40141                  7          1   
7                2        1819                  8          1   
8                2       43668                  9          0   
9                3       33754                  1          1   
10               3       24838                  2          1   
11               3       17704                  3          1   
12               3       21903                  4          1   
13               3       17668                  5          1   
14               3       46667          

In [10]:
prior_products_order['product_name'][0]

'Organic_Egg_Whites'

In [8]:
# add product name to each user
#each row correspons to one order. 
#The first column is each order_id. 
#The second column is the names of all products correspond to each order_id.

name_list = []
for i in prior_products_order.groupby('order_id')['product_name']:
    name_list.append(' '.join(i[1]))
    

In [9]:
#Order ID
order_id = prior_products_order.groupby('order_id')['product_name'].agg('count').index


In [10]:
#Creating a dataframe from list 
order_products_grouped = pd.DataFrame({'order_id':order_id, 'products':name_list})

In [14]:
order_products_grouped.head()

,order_id,products
0,2,Organic_Egg_Whites Michigan_Organic_Kale Garli...
1,3,Total_2%_with_Strawberry_Lowfat_Greek_Strained...
2,4,Plain_Pre-Sliced_Bagels Honey/Lemon_Cough_Drop...
3,5,"Bag_of_Organic_Bananas Just_Crisp,_Parmesan Fr..."
4,6,Cleanse Dryer_Sheets_Geranium_Scent Clean_Day_...


In [11]:
#As we are going to use PySpark to extract bigrams 
#we need to prepare the dataframe in a format required by PySpark

dataFrameList = []
index = 0
for row in order_products_grouped['products']:
    productsName = row.split(' ')
    tup = (index, productsName)
    dataFrameList.append(tup)
    index += 1

In [12]:
# randomly split data into train and test 
import random
import numpy
random.shuffle(dataFrameList)

train_data = dataFrameList[:2250411]
test_data = dataFrameList[2250411:]


In [13]:
#Extract Bigrams
#First convert the data to spark dataframe
#Using Word2Vec to convet the embedded words to pyspark format 
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec

spark = SparkSession.builder.appName("Bigram").getOrCreate()


In [14]:
# spark dataframe： read in batch of 10000 due to large computation
N = len(train_data) // 10000
mod = len(train_data) % 10000



In [15]:
trainDF = spark.createDataFrame(dataFrameList[0:10000] , ['id','product_name'])

In [16]:
#To reduce computation read 10000 lines each time, and then combine all into one spark dataframe.
for i in range(1,N):
    trainDF_sub = spark.createDataFrame(train_data[10000*i:10000*(i+1)], ['id',"product_name"])
    traintDF = trainDF.union(trainDF_sub)
    
trainDF_sub = spark.createDataFrame(train_data[10000*N:len(train_data)], ['id',"product_name"])
trainDF = trainDF.union(trainDF_sub)

In [17]:
# get bigram
#Using 2-Gram to do so 
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol="product_name", outputCol="bigrams")
ngramDataFrame = ngram.transform(trainDF)

ngramDataFrame.head()

Row(id=1495846, product_name=['Thirst_Quencher_Caffeine_Free_Naturally_Flavored_Citrus_Soda', 'Kosher_Salt', 'Iodized_Salt_&_Pepper', 'Diet_Vanilla_Root_Beer'], bigrams=['Thirst_Quencher_Caffeine_Free_Naturally_Flavored_Citrus_Soda Kosher_Salt', 'Kosher_Salt Iodized_Salt_&_Pepper', 'Iodized_Salt_&_Pepper Diet_Vanilla_Root_Beer'])

In [4]:
# count frequency:
# Bigrams are stored in a nested dictionary:
# first layer key is the first word in a bigram 
# second layer key is the second word in a bigram
# the second layer value is the frequency. 
# {'Organic_Mint_Bunch': {'Organic_Navel_Orange':2, 'c':2}}

bigrams = ngramDataFrame.toPandas()['bigrams']
table = {}
total = len(bigrams)
completion = 0
for bigram in bigrams:
    for combination in bigram:
        components = combination.split(' ')
        key = components[0]
        valKey = components[1]
        if key in table:
            valueDict = table[key]
            if valKey in valueDict:
                valueDict[valKey] = valueDict[valKey] + 1
            else:
                valueDict[valKey] = 1
        else:
            # create new value for key
            valueDict =  {valKey: 1}
            table[key] = valueDict
    completion += 1
print("==>", float(completion / total) * 100, "%")

NameError: name 'ngramDataFrame' is not defined

In [19]:
for firstWord in table:
    for secondWord in table[firstWord]:
        if table[firstWord][secondWord] > 20:
            print(firstWord, " + ", secondWord, ": ", table[firstWord][secondWord])

Banana  +  Organic_Strawberries :  25
Banana  +  Organic_Avocado :  33
Sparkling_Water_Grapefruit  +  Lime_Sparkling_Water :  22
Bag_of_Organic_Bananas  +  Organic_Strawberries :  26
Bag_of_Organic_Bananas  +  Organic_Raspberries :  27
Bag_of_Organic_Bananas  +  Organic_Hass_Avocado :  36
Organic_Avocado  +  Banana :  27
Organic_Hass_Avocado  +  Bag_of_Organic_Bananas :  27
Organic_Raspberries  +  Organic_Strawberries :  21
Organic_Strawberries  +  Organic_Blueberries :  26
Organic_Strawberries  +  Bag_of_Organic_Bananas :  21
Large_Lemon  +  Limes :  27


In [20]:
#Generate Recommendations
#It will use the frequencies above to generate recommendations for each product : make a function  

def getPureData(prodName):
    
#sort the bigram frequencies in descending order, 
#then return only the corresponding product names in the same order
    
    if prodName not in table:
        return []
    sortedOringalList = sorted(table[prodName].items(), key=lambda x: x[1], reverse=True)
#     print(sortedOringalList)
   
    data = {}
    for tp in sortedOringalList:
        product = tp[0]
        number = tp[1]
        if number in data:
            productList = data[number]
            productList.append(product)
        else:
            productList = [product]
        data[number] = productList
#     print(data)
#     print("==> Get pure data name:")
    
    pureData = data.values()
#     print(pureData)
    
    return list(pureData)

def pickRecommendProds(pureData, numberOfRecommend):
    
#Pick certain number of products from the sorted product names
    
    recommendProds = []
    for prods in pureData:
        if len(prods) <= numberOfRecommend:
            recommendProds += prods
            numberOfRecommend -= len(prods)
        else:
            recommendProds += random.sample(prods, numberOfRecommend)
            numberOfRecommend = 0

        if numberOfRecommend == 0:
            break
    
    return recommendProds

# recommend products bought together with 'name'
# name: the product to start with
def getRecommend(name, numberOfRecommend):
    
    
#Recommend certain number of products bought after the given input name
    
    # numberOfRecommend = 10
    recommendProducts = []
    productName = name
    index = 0

    while (numberOfRecommend):
#         print("->Target: ", productName)
#         print("->numberOfRecommend: ", numberOfRecommend)
#         print("->Index: ", index)
        data = getPureData(productName)
    #     print("Pure data:", data)
        intermediate = pickRecommendProds(data, numberOfRecommend)
        recommendProducts += intermediate
#         print("Recommend: ", recommendProducts)
#         print("Recommend: ", recommendProducts)
        if len(intermediate) == 0 and index == len(recommendProducts):
            break
        numberOfRecommend -= len(intermediate)
        if numberOfRecommend > 0:
#             print("Still left: ", numberOfRecommend)
            productName = recommendProducts[index]
            index += 1

#         print("==================")

    return recommendProducts

In [21]:
print(getRecommend("Organic_Mint_Bunch", 15))

['Large_Lemon', 'Original_Kombucha', 'Fresh_Cauliflower', 'Flat_Parsley,_Bunch', 'Grated_Parmesan', 'Organic_Hass_Avocado', 'Organic_Small_Bunch_Celery', 'Active_Dry_Yeast', 'Limes', 'Organic_Avocado', 'Banana', 'Cucumber_Kirby', 'Organic_Hass_Avocado', 'Organic_Dill', 'Organic_Baby_Spinach']


In [22]:
#We evaluate it by seeing in each order, how many products bought are in recommend list
#We compare the next 9 actually bought products with this 10 recommendations 
#If there's a match, we will add 1 to the total score 
#Then we move to the second actually bought product 
#and give another 10 recommendations bought with the second product. Compare again, 
#and compute total scores. After iterate through all actually bought products in this order
#we have the total score, and divide the score by the order size to get the final_score_1 for test order 1.

def TestScore(test_data):
    
    scores = []

    for order_info in test_data:
        this_order = order_info[1]
        order_len = len(this_order)
        #print('order:', this_order)
        #print('length of order', order_len)
        i = 0
        this_score = 0

        while (i < order_len):
            if this_order[i] in table:
                # use original order length as the num of recommendation
                recommends = getRecommend(this_order[i], order_len)
                #print('====> recommends of ', this_order[i], " : ", recommends)
                laterProds = this_order[i:]
                # check if the recommended products is included in order
                for prod in laterProds:
                    if prod in recommends:
                        #print("-->", prod)
                        this_score += 1
                i += 1
            else:
                # if the product is not trained in model, skip
                i += 1
                order_len -= 1

        #print(this_score)
        if not order_len == 0:
            scores.append(this_score/order_len)
        #print(scores)
        
    # return a list of predicted scores
    return(scores)

In [23]:
scores = TestScore(test_data)
print("======> Mean Test Scores: ", numpy.mean(scores))

======> Mean Test Scores:  0.18478416060614586


In [ ]:
#meaning in each of the order the products the customer bought are on an average 18% same as our recommendation list.
